# CAO Points Analysis

In [1]:
# Regular expressions
import re

# Package for making HTTP requests
import requests as rq

# Dates and times
import datetime as dt

# Data Frames
import pandas as pd

# For downloading
import urllib.request as urlrq

## 2021 Points

https://www.cao.ie/index.php?page=points&p=2021

In [2]:
#Fetch the CAO points URL
resp = rq.get('http://www2.cao.ie/points/l8.php')
# Have a look
resp

<Response [200]>

<br>

## Save original data set

In [3]:
#Get the current date and time.
now = dt.datetime.now()

# Format as a string
nowstr = now.strftime('%Y%m%d_%H%M%S')

In [4]:
#Create a file path for the original data
path = 'data/cao2021_' + nowstr + '.html'

<br>

## Charset error on server


Technically, server states decoding as:
    ```Content-Type: text/html; charset=iso-8859-1
    ```
However, one line uses \x96 which isn't defined in iso-8859-1.
Therefore, we use the similar decoding standard cp1252, which 
is very similar but includes \x96.

In [5]:
#The server uses the wrong encoding, fix it.
original_encoding = resp.encoding
#Change to cp1252
resp.encoding = 'cp1252'

In [6]:
#Save the original html file.
with open(path, 'w') as f:
    f.write(resp.text)

In [7]:
#Compile the regular expression for matching lines.
re_course = re.compile(r'([A-Z]{2}[0-9]{3})  (.*)([0-9]{3})(\*?) *')

In [8]:
# The file path for the csv file.
path = 'data/cao2021_csv_' + nowstr + '.csv'
    
#Loop throught the lines of the response content.
no_lines = 0

#Open the csv file for writing.
with open(path, 'w') as f:
    # Loop through lines of the response
    for line in resp.iter_lines():
        dline = line.decode('cp1252')
        # Match only the lines we want - the ones representing courses.
        if re_course.fullmatch(dline):
            #Add one to the lines counter
            no_lines = no_lines +1
            #Split the line on two or more spaces.
            linesplit = re.split('  +', dline)
            # Rejoin the substrings with commas in between           
            f.write(','.join(linesplit) + '\n')
            
# Print the total number of processed lines.            
print(f"Total number of lines is {no_lines}.")

Total number of lines is 922.


## 2020 Points
http://www.cao.ie/index.php?page=points&p=2020

<br>

#### Save original file

In [9]:
# Create a file path for the original data
path = 'data/cao2020_' + nowstr + '.xlsx'

In [10]:
#Save orihinal file to disk
urlrq.urlretrieve('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx', path)

('data/cao2020_20211204_164002.xlsx',
 <http.client.HTTPMessage at 0x1d2e527c130>)

<br>

#### Load spreadsheet using pandas

In [11]:
# Download and parse the excel spredsheet
df2020 = pd.read_excel('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx', skiprows = 10)

In [12]:
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Spot check last row
df.iloc[-1]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Mechanical and Manufacturing Engineering
COURSE CODE2                                                           WD230
R1 POINTS                                                                253
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      253
EOS Random *                                                             NaN
EOS Mid-point                                                            369
LEVEL                                                                      8
HEI                                        Waterford Institute of Technology
Test/Interview #                                                         NaN

In [14]:
# Create a file path for the pandas data
path = 'data/cao2020_' + nowstr + '.csv'

In [15]:
# Save pandas data frame to disk
df.to_csv(path)

<br>

## 2019 Points
http://www.cao.ie/index.php?page=points&p=2019

##### Steps to reproduce

1. Download original pdf file
2. Open original pdf file in Microsoft Word.
3. Save Microsoft Word's converted PDF in docx format.
4. Re-save Word document for editing.
5. Delete headers and footers.
6. Delete preamble on page 1.
7. Select all and copy.
8. Paste into Notepad ++.
9. Remeove HEI name headings and paste onto each cousre line.
10. Delete blank lines.
11. Delete tab characters at end of college groups.
12. Change all backticks to apostrophes

In [25]:
df2019 = pd.read_csv('data/cao2019_20211204_155130_edited.csv', sep = '\t')

In [26]:
df2019

,HEI,Course Code,INSTITUTION and COURSE,EOS,Mid
0,Athlone Institute of Technology,AL801,Software Design with Virtual Reality and Gaming,304,328.0
1,Athlone Institute of Technology,AL802,Software Design with Cloud Computing,301,306.0
2,Athlone Institute of Technology,AL803,Software Design with Mobile Apps and Connected...,309,337.0
3,Athlone Institute of Technology,AL805,Network Management and Cloud Infrastructure,329,442.0
4,Athlone Institute of Technology,AL810,Quantity Surveying,307,349.0
...,...,...,...,...,...
925,Waterford Institute of Technology,WD200,Arts (options),221,296.0
926,Waterford Institute of Technology,WD210,Software Systems Development,271,329.0
927,Waterford Institute of Technology,WD211,Creative Computing,275,322.0
928,Waterford Institute of Technology,WD212,Recreation and Sport Management,274,311.0
